In [1]:
# =============================================================================
# SPACEX FALCON 9 - INTERACTIVE VISUALIZATION WITH FOLIUM (ALL TASKS)
# IBM Data Science Capstone Project - PORTABLE VERSION
# =============================================================================

# =============================================================================
# CELL 1: INSTALL AND IMPORT LIBRARIES
# =============================================================================

import subprocess
import sys
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("SPACEX FALCON 9 - INTERACTIVE MAPS WITH FOLIUM")
print("="*80)

# Install required packages
required_packages = ['pandas', 'folium', 'branca']
for package in required_packages:
    try:
        __import__(package)
        print(f"✅ {package} already installed")
    except ImportError:
        print(f"📦 Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
        print(f"✅ {package} installed")

# Import libraries
import pandas as pd
import numpy as np
import folium
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon
import os
import urllib.request
from math import sin, cos, sqrt, atan2, radians
from branca.element import Figure

print("\n✅ All libraries imported successfully!")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🗺️ Folium version: {folium.__version__}")

# =============================================================================
# CELL 2: LOAD AND PREPARE SPACEX GEO DATA
# =============================================================================

print("\n" + "="*80)
print("📂 LOADING SPACEX LAUNCH GEO DATA")
print("="*80)

# Download dataset
data_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv"
local_path = "spacex_launch_geo.csv"

if not os.path.exists(local_path):
    print("📥 Downloading dataset...")
    urllib.request.urlretrieve(data_url, local_path)
    print("✅ Dataset downloaded")
else:
    print("✅ Dataset already exists locally")

# Load the data
spacex_df = pd.read_csv(local_path)
print(f"\n📊 Dataset shape: {spacex_df.shape}")
print(f"📊 Columns: {list(spacex_df.columns)}")

# Select relevant columns
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
print(f"\n📊 Selected columns: {list(spacex_df.columns)}")

# Create launch sites dataframe (unique sites)
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

print("\n📋 Launch Sites:")
print(launch_sites_df.to_string(index=False))

print(f"\n📊 Total launches: {len(spacex_df)}")
print(f"📊 Successful launches: {spacex_df['class'].sum()}")
print(f"📊 Failed launches: {len(spacex_df) - spacex_df['class'].sum()}")

# =============================================================================
# CELL 3: TASK 1 - MARK ALL LAUNCH SITES ON A MAP
# =============================================================================

print("\n" + "="*80)
print("🗺️ TASK 1: Marking All Launch Sites on Map")
print("="*80)

# NASA Johnson Space Center coordinates
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Create base map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
print("✅ Base map created")

# Add NASA JSC marker and circle
circle = folium.Circle(
    nasa_coordinate, 
    radius=1000, 
    color='#d35400', 
    fill=True
).add_child(folium.Popup('NASA Johnson Space Center'))

marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12pt; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
    )
)
site_map.add_child(circle)
site_map.add_child(marker)
print("✅ Added NASA JSC marker")

# Add circles and markers for each launch site
for index, site in launch_sites_df.iterrows():
    # Create circle
    circle = folium.Circle(
        [site['Lat'], site['Long']],
        radius=1000,
        color='#000000',
        fill=True
    ).add_child(folium.Popup(site['Launch Site']))
    
    # Create marker with site name
    marker = folium.map.Marker(
        [site['Lat'], site['Long']],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12pt; color:#d35400;"><b>{site["Launch Site"]}</b></div>',
        )
    )
    
    site_map.add_child(circle)
    site_map.add_child(marker)
    print(f"  ✅ Added {site['Launch Site']}")

print("\n✅ Task 1 complete: All launch sites marked")

# Display the map
site_map

# =============================================================================
# CELL 4: TASK 2 - MARK SUCCESS/FAILED LAUNCHES WITH CLUSTERS
# =============================================================================

print("\n" + "="*80)
print("🗺️ TASK 2: Marking Success/Failed Launches with Clusters")
print("="*80)

# Create marker_color column based on success/failure
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

print("\n📊 Launch outcome summary:")
print(f"  🟢 Success (green): {spacex_df['class'].sum()} launches")
print(f"  🔴 Failure (red): {len(spacex_df) - spacex_df['class'].sum()} launches")

# Create MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)
print("✅ MarkerCluster added")

# Add markers for each launch record
success_count = 0
failure_count = 0

for index, record in spacex_df.iterrows():
    # Determine icon color based on success/failure
    icon_color = 'green' if record['class'] == 1 else 'red'
    
    # Create popup text
    popup_text = f"""
    <b>Launch Site:</b> {record['Launch Site']}<br>
    <b>Outcome:</b> {'SUCCESS' if record['class'] == 1 else 'FAILURE'}<br>
    <b>Coordinates:</b> {record['Lat']:.4f}, {record['Long']:.4f}
    """
    
    # Create marker
    marker = folium.Marker(
        [record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=icon_color, icon='rocket', prefix='fa'),
        popup=folium.Popup(popup_text, max_width=300),
        tooltip=f"{record['Launch Site']} - {'Success' if record['class'] == 1 else 'Failure'}"
    )
    marker_cluster.add_child(marker)
    
    if record['class'] == 1:
        success_count += 1
    else:
        failure_count += 1

print(f"✅ Added {success_count} success markers (green)")
print(f"✅ Added {failure_count} failure markers (red)")
print("\n✅ Task 2 complete: All launches marked with clusters")

# Display the map
site_map

# =============================================================================
# CELL 5: TASK 3 - CALCULATE DISTANCES AND ADD PROXIMITY ANALYSIS
# =============================================================================

print("\n" + "="*80)
print("🗺️ TASK 3: Distance Calculations and Proximity Analysis")
print("="*80)

# Add MousePosition plugin to show coordinates
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)
print("✅ MousePosition plugin added (shows coordinates on hover)")

# Define distance calculation function
def calculate_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the distance between two points on Earth using Haversine formula
    """
    R = 6373.0  # Earth radius in kilometers
    
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    distance = R * c
    return distance

print("✅ Distance calculation function defined")

# Focus on CCAFS SLC-40 for proximity analysis
launch_site_lat = 28.563197
launch_site_lon = -80.576820
launch_site_name = "CCAFS SLC-40"

print(f"\n📍 Analyzing proximities for: {launch_site_name}")
print(f"   Coordinates: {launch_site_lat}°N, {launch_site_lon}°W")

# Find the closest coastline
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate distance to coastline
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create marker for coastline point
coastline_coordinate = [coastline_lat, coastline_lon]
coastline_marker = folium.Marker(
    coastline_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 10pt; color:#d35400;"><b>Coast<br>{distance_coastline:.2f} km</b></div>',
    ),
    popup=f"Coastline<br>Distance from launch site: {distance_coastline:.2f} km",
    tooltip="Click for details"
)
site_map.add_child(coastline_marker)
print(f"  ✅ Coastline marker added (distance: {distance_coastline:.2f} km)")

# Draw PolyLine between launch site and coastline
coastline_line = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    weight=2,
    color='blue',
    opacity=0.8,
    popup=f"Distance to Coast: {distance_coastline:.2f} km"
)
site_map.add_child(coastline_line)
print("  ✅ Coastline connection line added")

# Add marker for the launch site itself (for reference)
launch_site_marker = folium.Marker(
    [launch_site_lat, launch_site_lon],
    icon=folium.Icon(color='red', icon='info-sign'),
    popup=f"<b>{launch_site_name}</b><br>Reference point for distance calculations",
    tooltip=launch_site_name
)
site_map.add_child(launch_site_marker)

# Additional proximity analyses for various points of interest (NO EMOJIS)
proximities = {
    'Railway': [28.57114, -80.58542],
    'Highway': [28.56278, -80.57064],
    'Titusville City': [28.61212, -80.80757],
    'Airport': [28.51831, -80.79891],
    'Industrial Area': [28.56912, -80.57628],
    'Cocoa Beach': [28.3187, -80.6078],
    'KSC Visitors Center': [28.5231, -80.6819],
    'Cape Canaveral Lighthouse': [28.4608, -80.5275]
}

print("\n📊 Adding proximity markers:")

# Add markers and lines for each proximity point
for name, coord in proximities.items():
    distance = calculate_distance(launch_site_lat, launch_site_lon, coord[0], coord[1])
    
    # Create marker with custom icon
    marker = folium.Marker(
        coord,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 9pt; color:#3186cc; background-color:white; padding:2px; border-radius:3px; border:1px solid #3186cc;"><b>{name}<br>{distance:.2f} km</b></div>',
        ),
        popup=f"<b>{name}</b><br>Distance from {launch_site_name}: {distance:.2f} km<br>Coordinates: {coord[0]:.4f}, {coord[1]:.4f}",
        tooltip=f"{name} - {distance:.2f} km"
    )
    site_map.add_child(marker)
    
    # Create connecting line
    line = folium.PolyLine(
        locations=[[launch_site_lat, launch_site_lon], coord],
        weight=1,
        color='red',
        opacity=0.4,
        popup=f"Distance to {name}: {distance:.2f} km"
    )
    site_map.add_child(line)
    
    print(f"  ✅ {name}: {distance:.2f} km")

# Add a title to the map (NO EMOJIS)
title_html = '''
<div style="position: fixed; top: 10px; left: 50px; width: 300px; height: 50px; 
            background-color: white; border-radius: 5px; padding: 10px; 
            border: 2px solid grey; z-index: 9999; font-size: 14px;">
    <b>SpaceX Launch Sites & Proximity Analysis</b><br>
    <span style="color: green;">Green = Success</span> | 
    <span style="color: red;">Red = Failure</span> | 
    <span style="color: blue;">Blue = Coastline</span>
</div>
'''
site_map.get_root().html.add_child(folium.Element(title_html))

print("\n✅ Task 3 complete: All proximity markers and lines added")

# Display the final map
site_map

# =============================================================================
# CELL 6: PRINT ANALYSIS FINDINGS
# =============================================================================

print("\n" + "="*80)
print("📊 LAUNCH SITE PROXIMITY ANALYSIS - RESULTS")
print("="*80)

print(f"\n📍 Launch Site: {launch_site_name} ({launch_site_lat}°N, {launch_site_lon}°W)")
print("-" * 50)

# Calculate all distances for display
distances = {
    'Coastline': calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
}

for name, coord in proximities.items():
    distances[name] = calculate_distance(launch_site_lat, launch_site_lon, coord[0], coord[1])

# Sort distances
sorted_distances = sorted(distances.items(), key=lambda x: x[1])

print("\n📏 Distances from launch site (sorted):")
for name, distance in sorted_distances:
    print(f"  {name:20s}: {distance:.2f} km")

print("\n" + "="*80)
print("🔍 FINDINGS AND OBSERVATIONS")
print("="*80)

findings = [
    "1. Coastal Proximity: All launch sites are very close to the coast (within ~1 km)",
    "   -> Essential for safety: rockets travel over water, not populated areas",
    "",
    "2. Infrastructure Access: Launch sites have excellent access to:",
    "   -> Railways: For transporting rocket components (within 2-3 km)",
    "   -> Highways: For ground transportation (within 1-2 km)",
    "   -> Nearby cities: Workforce accommodation (Titusville ~24 km)",
    "   -> Airports: Logistics and personnel travel (Airport ~25 km)",
    "",
    "3. Equator Proximity: Florida sites at ~28.5°N latitude",
    "   -> Provides rotational speed boost for rockets (~400 m/s advantage)",
    "",
    "4. Safety Considerations:",
    "   -> Sites maintain reasonable distance from densely populated areas",
    "   -> Clear flight corridors over Atlantic Ocean",
    "   -> Multiple landing zones (drone ships, ground pads) available",
    "",
    "5. CCAFS SLC-40 Specific:",
    f"   -> Closest to coastline: {distances['Coastline']:.2f} km",
    f"   -> Railway access: {distances['Railway']:.2f} km",
    f"   -> Highway access: {distances['Highway']:.2f} km",
]

for finding in findings:
    print(finding)

print("\n" + "="*80)

# =============================================================================
# CELL 7: CREATE ADDITIONAL MAPS FOR EACH LAUNCH SITE
# =============================================================================

print("\n" + "="*80)
print("🗺️ CREATING INDIVIDUAL MAPS FOR EACH LAUNCH SITE")
print("="*80)

# Dictionary to store individual site maps
site_maps = {}

for idx, site in launch_sites_df.iterrows():
    site_name = site['Launch Site']
    site_lat = site['Lat']
    site_lon = site['Long']
    
    print(f"\n📍 Creating map for {site_name}...")
    
    # Create map centered on this site
    site_map_detail = folium.Map(location=[site_lat, site_lon], zoom_start=12)
    
    # Add all launches for this site with markers
    site_launches = spacex_df[spacex_df['Launch Site'] == site_name]
    
    # Add a circle for the site
    folium.Circle(
        [site_lat, site_lon],
        radius=2000,
        color='#3186cc',
        fill=True,
        fillOpacity=0.2,
        popup=f"<b>{site_name}</b><br>Launch Site"
    ).add_to(site_map_detail)
    
    # Add markers for each launch
    for _, launch in site_launches.iterrows():
        color = 'green' if launch['class'] == 1 else 'red'
        folium.Marker(
            [launch['Lat'], launch['Long']],
            icon=folium.Icon(color=color, icon='rocket', prefix='fa'),
            popup=f"Launch Outcome: {'Success' if launch['class'] == 1 else 'Failure'}",
            tooltip=f"{'Success' if launch['class'] == 1 else 'Failure'}"
        ).add_to(site_map_detail)
    
    # Add title
    title = f'<div style="position: fixed; top: 10px; left: 50px; background-color: white; padding: 5px; border-radius: 5px; border: 1px solid grey; z-index: 9999;"><b>{site_name} - Launch Details</b></div>'
    site_map_detail.get_root().html.add_child(folium.Element(title))
    
    site_maps[site_name] = site_map_detail
    print(f"  ✅ Map created with {len(site_launches)} launches")

print("\n✅ Individual site maps created")

# =============================================================================
# CELL 8: SAVE MAPS TO LOCAL FOLDER (OPTIONAL - COMMENT OUT IF NOT NEEDED)
# =============================================================================

# Uncomment the following section if you want to save the maps locally

print("\n" + "="*80)
print("💾 SAVING MAPS TO LOCAL FOLDER (OPTIONAL)")
print("="*80)

# Create a local folder for maps
maps_folder = "folium_maps"
os.makedirs(maps_folder, exist_ok=True)
print(f"✅ Created folder: {maps_folder}")

# Save main interactive map as HTML
main_map_path = os.path.join(maps_folder, "all_launch_sites_map.html")
site_map.save(main_map_path)
print(f"  ✅ Saved: all_launch_sites_map.html")

# Create a simple HTML instructions file
html_instructions = """
<html>
<head><title>SpaceX Launch Maps</title></head>
<body style="font-family: Arial, sans-serif; margin: 20px;">
    <h1>SpaceX Falcon 9 Launch Maps</h1>
    <p>These are interactive Folium maps. Open the HTML files in your browser to explore:</p>
    <ul>
        <li><a href="all_launch_sites_map.html">All Launch Sites Map</a> - Complete map with all launches</li>
        <li><a href="ccafs_slc_40_map.html">CCAFS SLC-40 Map</a> - Detailed view with proximity analysis</li>
        <li><a href="ksc_lc_39a_map.html">KSC LC-39A Map</a> - Kennedy Space Center launches</li>
        <li><a href="vafb_slc_4e_map.html">VAFB SLC-4E Map</a> - Vandenberg launches</li>
    </ul>
    <p>To save as PNG: Take a screenshot of each map in your browser</p>
</body>
</html>
"""

index_path = os.path.join(maps_folder, "index.html")
with open(index_path, 'w', encoding='utf-8') as f:
    f.write(html_instructions)
print(f"  ✅ Saved: index.html (navigation page)")

# Save individual site maps
for site_name, site_map_obj in site_maps.items():
    # Clean filename (remove special characters)
    filename = site_name.lower().replace(' ', '_').replace('-', '_').replace('&', 'and')
    map_path = os.path.join(maps_folder, f"{filename}_map.html")
    site_map_obj.save(map_path)
    print(f"  ✅ Saved: {filename}_map.html")

# Create a summary text file with distances
summary_path = os.path.join(maps_folder, "distance_analysis.txt")
with open(summary_path, 'w', encoding='utf-8') as f:
    f.write("="*60 + "\n")
    f.write("SPACEX LAUNCH SITE PROXIMITY ANALYSIS\n")
    f.write("="*60 + "\n\n")
    f.write(f"Launch Site: {launch_site_name}\n")
    f.write(f"Coordinates: {launch_site_lat}°N, {launch_site_lon}°W\n\n")
    f.write("Distances from launch site:\n")
    f.write("-"*40 + "\n")
    for name, distance in sorted_distances:
        f.write(f"{name:20s}: {distance:.2f} km\n")
    f.write("\n" + "="*60 + "\n")
    f.write("FINDINGS:\n")
    f.write("="*60 + "\n")
    f.write("1. All launch sites within 1 km of coastline for safety\n")
    f.write("2. Excellent railway and highway access for logistics\n")
    f.write("3. Reasonable distance from populated areas (Titusville ~24 km)\n")
    f.write("4. Florida sites at ~28.5°N provide rotational speed boost\n")
print(f"  ✅ Saved: distance_analysis.txt")

print(f"\n✅ All maps saved to: {maps_folder}/")
print("\n📁 Files created:")
print(f"  📄 {maps_folder}/all_launch_sites_map.html - Main interactive map")
print(f"  📄 {maps_folder}/index.html - Navigation page")
print(f"  📄 {maps_folder}/ccafs_slc_40_map.html - CCAFS detailed map")
print(f"  📄 {maps_folder}/ksc_lc_39a_map.html - KSC detailed map")
print(f"  📄 {maps_folder}/vafb_slc_4e_map.html - VAFB detailed map")
print(f"  📄 {maps_folder}/distance_analysis.txt - Distance calculations")

print("\n💡 TIP: Open the HTML files in your browser to explore interactive maps")
print("💡 To save as PNG: Take screenshots of each map in your browser")

# =============================================================================
# CELL 9: DISPLAY FINAL SUMMARY
# =============================================================================

print("\n" + "="*80)
print("✅ FOLIUM INTERACTIVE VISUALIZATION COMPLETED SUCCESSFULLY!")
print("="*80)

print(f"""
📊 SUMMARY STATISTICS:
   • Total launches analyzed: {len(spacex_df)}
   • Successful launches: {spacex_df['class'].sum()} ({(spacex_df['class'].mean()*100):.1f}%)
   • Failed launches: {len(spacex_df) - spacex_df['class'].sum()} ({(1-spacex_df['class'].mean())*100:.1f}%)
   • Launch sites: {len(launch_sites_df)}
   • Proximity points analyzed: {len(proximities) + 1} (including coastline)

🗺️ MAPS GENERATED:
   • Main map: All launch sites with all launches
   • Individual maps for each launch site
   • Proximity analysis for CCAFS SLC-40

📁 MAPS SAVED TO: (if save option was enabled)
   • Local folder: folium_maps/

📌 NEXT STEPS:
   1. Open the HTML files in your browser (if saved)
   2. Explore interactive features (zoom, click markers, hover)
   3. Take screenshots for your reports
   4. Use distance analysis for your findings section
""")

print("="*80)

SPACEX FALCON 9 - INTERACTIVE MAPS WITH FOLIUM
✅ pandas already installed
✅ folium already installed
✅ branca already installed

✅ All libraries imported successfully!
📊 Pandas version: 3.0.1
🗺️ Folium version: 0.20.0

📂 LOADING SPACEX LAUNCH GEO DATA
📥 Downloading dataset...
✅ Dataset downloaded

📊 Dataset shape: (56, 13)
📊 Columns: ['Flight Number', 'Date', 'Time (UTC)', 'Booster Version', 'Launch Site', 'Payload', 'Payload Mass (kg)', 'Orbit', 'Customer', 'Landing Outcome', 'class', 'Lat', 'Long']

📊 Selected columns: ['Launch Site', 'Lat', 'Long', 'class']

📋 Launch Sites:
 Launch Site       Lat        Long
 CCAFS LC-40 28.562302  -80.577356
CCAFS SLC-40 28.563197  -80.576820
  KSC LC-39A 28.573255  -80.646895
 VAFB SLC-4E 34.632834 -120.610745

📊 Total launches: 56
📊 Successful launches: 24
📊 Failed launches: 32

🗺️ TASK 1: Marking All Launch Sites on Map
✅ Base map created
✅ Added NASA JSC marker
  ✅ Added CCAFS LC-40
  ✅ Added CCAFS SLC-40
  ✅ Added KSC LC-39A
  ✅ Added VAFB SLC